# ETL + Connection

In [31]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import pyodbc
print(pyodbc.drivers())
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, Date as SqlDate
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import json

['SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)']


In [32]:
airbnb = pd.read_csv("Data/AIRBNB.csv", sep=';', low_memory=False)
arrest = pd.read_csv("Data/ARREST.csv", sep=";", low_memory=False)
arrest['ARREST_DATE'] = pd.to_datetime(arrest['ARREST_DATE'], format='%m/%d/%Y')

# Percent of the data
percent = 5

# How many districts (num of districts = n^2)
n = 120

# Connection arguments
with open('password.txt', 'r') as file:
    data = json.load(file)
    passwordJSON = next(iter(data.values()))

server = 'LAPTOP-SHQVUF0B\MSSQLFINAL'
database = 'AirBNB&Arrest'
username = 'sa'
password = passwordJSON

## AIRBNB

In [33]:
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def is_valid_lat(value):
    try:
        return 33 <= float(value) <= 47
    except ValueError:
        return False

def is_valid_long(value):
    try:
        return -80 <= float(value) <= -50
    except ValueError:
        return False

# Function to clean the dataframe
def clean_airbnb_data(df):
    for index, row in df.iterrows():
        # Check lat and long for non-float values and move them to neighbourhood/neighbourhood_group
        for col in ['lat', 'long']:
            if col == 'lat' and not is_valid_lat(row[col]):
                if pd.notnull(row[col]):
                    value = str(row[col])
                    if value in df['neighbourhood group'].unique():
                        df.at[index, 'neighbourhood group'] = value
                    elif value in df['neighbourhood'].unique():
                        df.at[index, 'neighbourhood'] = value
                    df.at[index, col] = np.nan
            elif col == 'long' and not is_valid_long(row[col]):
                if pd.notnull(row[col]):
                    value = str(row[col])
                    if value in df['neighbourhood group'].unique():
                        df.at[index, 'neighbourhood group'] = value
                    elif value in df['neighbourhood'].unique():
                        df.at[index, 'neighbourhood'] = value
                    df.at[index, col] = np.nan

        # Check neighbourhood and neighbourhood_group for numeric values and move them to lat/long
        if is_float(row['neighbourhood group']):
            value = float(row['neighbourhood group'])
            if is_valid_lat(value):
                df.at[index, 'lat'] = value
            elif is_valid_long(value):
                df.at[index, 'long'] = value
            df.at[index, 'neighbourhood group'] = np.nan

        if is_float(row['neighbourhood']):
            value = float(row['neighbourhood'])
            if is_valid_lat(value):
                df.at[index, 'lat'] = value
            elif is_valid_long(value):
                df.at[index, 'long'] = value
            df.at[index, 'neighbourhood'] = np.nan

    # Fill missing lat/long values with the mean of the column for the corresponding neighbourhood and neighbourhood_group
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['long'] = pd.to_numeric(df['long'], errors='coerce')

    for index, row in df.iterrows():
        if pd.isnull(row['lat']) or pd.isnull(row['long']):
            group = row['neighbourhood group']
            hood = row['neighbourhood']
            if pd.isnull(row['lat']):
                mean_lat = df[(df['neighbourhood group'] == group) & (df['neighbourhood'] == hood)]['lat'].mean()
                df.at[index, 'lat'] = mean_lat
            if pd.isnull(row['long']):
                mean_long = df[(df['neighbourhood group'] == group) & (df['neighbourhood'] == hood)]['long'].mean()
                df.at[index, 'long'] = mean_long

    return df


def convert(airbnb):
    # Convert columns to appropriate data types
    airbnb['ID'] = pd.to_numeric(airbnb['ID'], errors='coerce').astype('Int64')
    airbnb['Host ID'] = pd.to_numeric(airbnb['Host ID'], errors='coerce').astype('Int64')
    airbnb['Instant Bookable'] = airbnb['Instant Bookable'].apply(lambda x: True if str(x).lower() == 'true' else False)
    airbnb['Construction Year'] = pd.to_numeric(airbnb['Construction Year'], errors='coerce').astype('Int64')
    airbnb['Price'] = airbnb['Price'].replace('[\$,]', '', regex=True)
    airbnb['Service Fee'] = airbnb['Service Fee'].replace('[\$,]', '', regex=True)
    airbnb['Minimum Nights'] = pd.to_numeric(airbnb['Minimum Nights'], errors='coerce').astype('Int64')
    airbnb['Number of Reviews'] = pd.to_numeric(airbnb['Number of Reviews'], errors='coerce').astype('Int64')
    airbnb['Last Review'] = pd.to_datetime(airbnb['Last Review'], errors='coerce')
    airbnb['Reviews per Month'] = pd.to_numeric(airbnb['Reviews per Month'], errors='coerce').astype(float)
    airbnb['Review Rate Number'] = pd.to_numeric(airbnb['Review Rate Number'], errors='coerce').apply(lambda x: x if x in [1, 2, 3, 4, 5, 6] else pd.NA).astype('Int64')
    airbnb['Calculated Host Listings Count'] = pd.to_numeric(airbnb['Calculated Host Listings Count'], errors='coerce').astype('Int64')
    airbnb['Availability 365'] = pd.to_numeric(airbnb['Availability 365'], errors='coerce').astype('Int64')

    # Convert Price and Service Fee to float
    airbnb['Price'] = pd.to_numeric(airbnb['Price'], errors='coerce')
    airbnb['Service Fee'] = pd.to_numeric(airbnb['Service Fee'], errors='coerce')

    # List of columns that cannot have null values
    columns_to_check = ['ID', 'Name', 'Host ID', 'Host Identity Verified', 'Host Name', 'Neighbourhood Group',
                        'Neighbourhood', 'Latitude', 'Longitude', 'Instant Bookable', 'Cancellation Policy',
                        'Room Type', 'Construction Year', 'Price', 'Service Fee', 'Minimum Nights', 'Number of Reviews',
                        'Reviews per Month', 'Review Rate Number', 'Availability 365']

    # Drop rows with any NaN values in the specified columns
    airbnb.dropna(subset=columns_to_check, inplace=True)

    return airbnb


airbnb = clean_airbnb_data(airbnb)
airbnb.columns = [
    'ID', 'Name', 'Host ID', 'Host Identity Verified', 'Host Name', 'Neighbourhood Group', 'Neighbourhood', 'Latitude',
    'Longitude', 'Instant Bookable', 'Cancellation Policy', 'Room Type', 'Construction Year', 'Price', 'Service Fee',
    'Minimum Nights', 'Number of Reviews', 'Last Review', 'Reviews per Month', 'Review Rate Number',
    'Calculated Host Listings Count', 'Availability 365'
]
airbnb = convert(airbnb)

## ARREST

In [34]:
def is_valid_lat(value):
    try:
        return 33 <= float(value) <= 47
    except ValueError:
        return False

def is_valid_long(value):
    try:
        return -80 <= float(value) <= -50
    except ValueError:
        return False

arrest = arrest[arrest.apply(lambda row: is_valid_lat(row['Latitude']) and is_valid_long(row['Longitude']), axis=1)]


In [35]:
arrest.to_csv("ARRESTProper.csv", index = False)
airbnb.to_csv("AIRBNBProper.csv", index = False)

# DATA WAREHOUSE CREATION

In [36]:
airbnb = pd.read_csv('AIRBNBProper.csv')
arrest = pd.read_csv('ARRESTProper.csv')

In [37]:
airbnb = airbnb.sample(n=int(len(airbnb)*percent/100), random_state=42)
arrest = arrest.sample(n=int(len(arrest)*percent/100), random_state=42)

# Data frames had such maximum values, which were used to limit the geographical space.
lat_min, lat_max = 40.501, 40.917
long_min, long_max = -74.254, -73.705

# Calculation of the width and height of one smaller rectangle
lat_step = (lat_max - lat_min) / n
long_step = (long_max - long_min) / n

# Creating IDs for rectangles and average latitudes and longitudes
rectangles = []
for i in range(n):
    for j in range(n):
        rect_id = i * n + j
        lat_center = lat_min + (i + 0.5) * lat_step
        long_center = long_min + (j + 0.5) * long_step
        lat_bounds = (lat_min + i * lat_step, lat_min + (i + 1) * lat_step)
        long_bounds = (long_min + j * long_step, long_min + (j + 1) * long_step)
        rectangles.append((rect_id, lat_center, long_center, lat_bounds, long_bounds))

# Function assigning ID and averaged coordinate values to a row
def assign_rectangle_id(row, rectangles):
    lat = row['Latitude']
    long = row['Longitude']
    for rect_id, lat_center, long_center, lat_bounds, long_bounds in rectangles:
        if lat_bounds[0] <= lat < lat_bounds[1] and long_bounds[0] <= long < long_bounds[1]:
            return pd.Series([rect_id, lat_center, long_center])

# Assigning IDs and coordinates for the 'arrest' dataframe
arrest[['Rectangle_ID', 'Avg_Latitude', 'Avg_Longitude']] = arrest.apply(lambda row: assign_rectangle_id(row, rectangles), axis=1)

# Assigning IDs and coordinates for the 'airbnb' dataframe
airbnb[['Rectangle_ID', 'Avg_Latitude', 'Avg_Longitude']] = airbnb.apply(lambda row: assign_rectangle_id(row, rectangles), axis=1)


## Final creation

In [38]:
arrest_df = arrest
airbnb_df = airbnb

arrest_dates = pd.to_datetime(arrest_df['ARREST_DATE'])
if 'Last Review' in airbnb_df.columns:
    airbnb_dates = pd.to_datetime(airbnb_df['Last Review'], errors='coerce')
elif 'LastReview' in airbnb_df.columns:
    airbnb_dates = pd.to_datetime(airbnb_df['LastReview'], errors='coerce')

all_dates = pd.concat([arrest_dates, airbnb_dates]).dropna().reset_index(drop=True)
all_dates = pd.to_datetime(all_dates)

# Create the date dataframe
date_df = pd.DataFrame({
    'Date_ID': all_dates,
    'Year': all_dates.dt.year,
    'Month': all_dates.dt.month,
    'Week': all_dates.dt.isocalendar().week,
    'Weekday': all_dates.dt.day_name(),
    'Weekend': all_dates.dt.weekday >= 5,
    'Day': all_dates.dt.day
})

date_df = date_df.drop_duplicates().reset_index(drop=True)

# Create the CrimeCategory dataframe
crime_category_df = arrest_df[['OFNS_DESC', 'LAW_CAT_CD']].drop_duplicates().reset_index(drop=True)
crime_category_df.index.name = 'CrimeCategory_ID'
crime_category_df.reset_index(inplace=True)
crime_category_df.rename(columns={'OFNS_DESC': 'OffenseDescription', 'LAW_CAT_CD': 'LawCategory'}, inplace=True)

# Create the Arrest dataframe
arrest_df['Date_ID'] = pd.to_datetime(arrest_df['ARREST_DATE'])
arrest_df['CrimeCategory_ID'] = arrest_df.groupby(['OFNS_DESC', 'LAW_CAT_CD']).ngroup()
arrest_df['AIRBNB_ID'] = None  # Placeholder as AIRBNB_ID is not in the provided arrest_df

arrest_df = arrest_df.rename(columns={
    'ARREST_KEY': 'Arrest_ID',
    'ARREST_BORO': 'ArrestBorough',
    'AGE_GROUP': 'AgeGroup',
    'PERP_SEX': 'Sex',
    'PERP_RACE': 'Race'
})[['Arrest_ID', 'Date_ID', 'CrimeCategory_ID', 'ArrestBorough', 'AgeGroup', 'Sex', 'Race', 'Latitude', 'Longitude', 'Rectangle_ID', 'Avg_Longitude', 'Avg_Latitude']]

# Create the AIRBNB dataframe
airbnb_df.rename(columns={
    'ID': 'AIRBNB_ID',
    'Host ID': 'Host_ID',
    'Neighbourhood Group': 'Neighbourhood_ID',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Instant Bookable': 'InstantBookable',
    'Cancellation Policy': 'CancellationPolicy',
    'Room Type': 'RoomType',
    'Construction Year': 'ConstructionYear',
    'Service Fee': 'ServiceFee',
    'Minimum Nights': 'MinimumNights',
    'Number of Reviews': 'NumberOfReviews',
    'Last Review': 'LastReview',
    'Reviews per Month': 'ReviewsPerMonth',
    'Review Rate Number': 'ReviewRate',
    'Calculated Host Listings Count': 'HostListingsCount',
    'Availability 365': 'Availability365'
}, inplace=True)
airbnb_df.drop(columns=['Neighbourhood_ID'], inplace=True)
airbnb_df.rename(columns={'Neighbourhood': 'Neighbourhood_ID'}, inplace=True)

# Create the Host dataframe
host_df = airbnb_df[['Host_ID', 'Host Name', 'Host Identity Verified', 'HostListingsCount']].drop_duplicates().reset_index(drop=True)
host_df.rename(columns={
    'Host_ID': 'Host_ID',
    'Host Name': 'HostName',
    'Host Identity Verified': 'Verification',
    'HostListingsCount': 'HostListingsCount'
}, inplace=True)

# Create the Neighbourhood dataframe
neighbourhood_df = airbnb_df[['Neighbourhood_ID']].drop_duplicates().reset_index(drop=True)
neighbourhood_df['Group'] = neighbourhood_df['Neighbourhood_ID']


arrest_df['Rectangle_ID'] = arrest_df['Rectangle_ID'].astype('Int64')  # Using 'Int64' to handle possible NaNs
arrest_df['CrimeCategory_ID'] = arrest_df['CrimeCategory_ID'].astype('Int64')

# 2. Convert 'Neighbourhood_ID' and 'Rectangle_ID' to int and 'LastReview' to datetime in airbnb_df
airbnb_df['Neighbourhood_ID'] = airbnb_df['Neighbourhood_ID']
airbnb_df['Rectangle_ID'] = airbnb_df['Rectangle_ID'].astype('Int64')
airbnb_df['LastReview'] = pd.to_datetime(airbnb_df['LastReview'], errors='coerce')  # Convert to datetime, coerce errors to NaT

# 3. Convert remaining 'object' types to 'string' in all DataFrames
# Define a helper function to convert object columns to string
def convert_object_to_string(df):
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].astype(str)
    return df

date_df = convert_object_to_string(date_df)
crime_category_df = convert_object_to_string(crime_category_df)
host_df = convert_object_to_string(host_df)
neighbourhood_df = convert_object_to_string(neighbourhood_df)
arrest_df = convert_object_to_string(arrest_df)
arrest_df = arrest_df.dropna()
airbnb_df = convert_object_to_string(airbnb_df)
airbnb_df = airbnb_df.drop_duplicates(subset=['AIRBNB_ID'])

In [39]:
arrest_df.to_csv("FinalData/ARREST.csv", index = False)
date_df.to_csv("FinalData/DATE.csv", index = False)
neighbourhood_df.to_csv("FinalData/NEIGHBOURHOOD.csv", index = False)
host_df.to_csv("FinalData/HOST.csv", index = False)
airbnb_df.to_csv("FinalData/AIRBNB.csv", index = False)
crime_category_df.to_csv("FinalData/CRIME_CATEGORY.csv", index = False)

# SQL CONNECT

In [40]:
airbnb = pd.read_csv('FinalData/AIRBNB.csv')
arrest = pd.read_csv('FinalData/ARREST.csv')
crime_category = pd.read_csv('FinalData/CRIME_CATEGORY.csv')
date = pd.read_csv('FinalData/DATE.csv')
host = pd.read_csv('FinalData/HOST.csv')
neighbourhood = pd.read_csv('FinalData/NEIGHBOURHOOD.csv')

In [41]:
# Function to test connection
def test_connection():
    try:
        conn = pyodbc.connect(
            f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        )
        print("Połączenie udane!")
        conn.close()
    except pyodbc.Error as ex:
        sqlstate = ex.args[1]
        print(f"Błąd połączenia: {sqlstate}")

# Testowanie połączenia
test_connection()

# Tworzenie połączenia z SQL Server za pomocą SQLAlchemy
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')
Base = declarative_base()

# Defining the schema for the tables
class Airbnb(Base):
    __tablename__ = 'airbnb'
    AIRBNB_ID = Column(Integer, primary_key=True)
    Name = Column(String)
    Host_ID = Column(Integer)
    Host_Identity_Verified = Column(String)
    Host_Name = Column(String)
    Neighbourhood_ID = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    InstantBookable = Column(Boolean)
    CancellationPolicy = Column(String)
    RoomType = Column(String)
    ConstructionYear = Column(Integer)
    Price = Column(Float)
    ServiceFee = Column(Float)
    MinimumNights = Column(Integer)
    NumberOfReviews = Column(Integer)
    LastReview = Column(SqlDate)
    ReviewsPerMonth = Column(Float)
    ReviewRate = Column(Integer)
    HostListingsCount = Column(Float)
    Availability365 = Column(Integer)
    Rectangle_ID = Column(Integer)
    Avg_Latitude = Column(Float)
    Avg_Longitude = Column(Float)

class Arrest(Base):
    __tablename__ = 'arrest'
    Arrest_ID = Column(Integer, primary_key=True)
    Date_ID = Column(String)
    CrimeCategory_ID = Column(Integer)
    ArrestBorough = Column(String)
    AgeGroup = Column(String)
    Sex = Column(String)
    Race = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Rectangle_ID = Column(Integer)
    Avg_Longitude = Column(Float)
    Avg_Latitude = Column(Float)

class CrimeCategory(Base):
    __tablename__ = 'crime_category'
    CrimeCategory_ID = Column(Integer, primary_key=True)
    OffenseDescription = Column(String)
    LawCategory = Column(String)

class DateTable(Base):
    __tablename__ = 'date'
    Date_ID = Column(String(255), primary_key=True)
    Year = Column(Integer)
    Month = Column(Integer)
    Week = Column(Integer)
    Weekday = Column(String)
    Weekend = Column(Boolean)
    Day = Column(Integer)

class Host(Base):
    __tablename__ = 'host'
    Host_ID = Column(Integer, primary_key=True)
    HostName = Column(String)
    Verification = Column(String)
    HostListingsCount = Column(Float)

class Neighbourhood(Base):
    __tablename__ = 'neighbourhood'
    Neighbourhood_ID = Column(String(255), primary_key=True)
    Group = Column(String)

# Creating the tables in the database
Base.metadata.create_all(engine)

def load_data_to_sql(df, table_name):
    df = df.drop_duplicates(subset=[df.columns[0]])
    temp_table_name = table_name + "_temp"
    df.to_sql(temp_table_name, con=engine, if_exists='replace', index=False)
    
    with engine.connect() as connection:
        primary_key_column = df.columns[0]
        delete_query = f"""
        DELETE FROM {table_name}
        WHERE {primary_key_column} IN (SELECT {primary_key_column} FROM {temp_table_name});
        """
        connection.execute(delete_query)
    
        insert_query = f"""
        INSERT INTO {table_name}
        SELECT * FROM {temp_table_name};
        """
        connection.execute(insert_query)
        
        connection.execute(f"DROP TABLE {temp_table_name};")

# Converting LastReview to datetime format
airbnb['LastReview'] = pd.to_datetime(airbnb['LastReview'], errors='coerce')

# Loading the data into SQL Server
load_data_to_sql(airbnb, 'airbnb')
load_data_to_sql(arrest, 'arrest')
load_data_to_sql(crime_category, 'crime_category')
load_data_to_sql(date, 'date')
load_data_to_sql(host, 'host')
load_data_to_sql(neighbourhood, 'neighbourhood')

print("Dane zostały pomyślnie załadowane do SQL Server.")

Połączenie udane!
Dane zostały pomyślnie załadowane do SQL Server.
